# Archive scene cost estimate

---
Here we will estimate the cost in credits to order scenes identified in our catalog search. This notebook assumes you have already run through the catalog-search notebook as we will be using the results that were exported to our search-report geojson file. </br>
The intent of separating these different steps into unique notebooks is so that we can focus on distinct capabilities of the UP42 API but also to keep the notebooks from becoming lengthy.</br>
---

For the estimate we will need to bring some parameters together and pass them to the construct_order_parameters() function. We will need three items: </br>
1. aoi - The geometry of the order</br>
2. image_id - The id of the desired image (from search results) </br>
3. data_product_id - Unique identifier of the desired UP42 data product, see catalog.get_data_products (i.e. Pleiades display, WorldDEM Neo, etc..)

The construct_order_parameters() function builds a json payload that the UP42 API will use to return an estimate for the cost of the scene overlapping the AOI provided. 

## Python Libraries

In [1]:
import up42

from pathlib import Path
from pprint import pprint

## Authenticate with UP42

This notebook is meant as a following step to the archive search that produced by search report exported to geojson format. If you have not looked at the catalog-searcg notebook please do so now before moving on with this notebook.

In [2]:
# Project credentials
project_conf_file = Path.cwd()/'proj-conf-file.json'
up42.authenticate(cfg_file = project_conf_file)

2023-02-09 15:20:51,943 - Got credentials from config file.
2023-02-09 15:20:52,942 - Authentication with UP42 successful!


In [3]:
# Initialze catalog
catalog = up42.initialize_catalog()

## Credit estimate

### AOI 

In [4]:
# Load aoi from file

geojson_aoi_file = Path.cwd()/'aoi/aoi-teste-de-buch-fire.geojson'
aoi = up42.read_vector_file(geojson_aoi_file)
aoi

{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {},
   'geometry': {'type': 'Polygon',
    'coordinates': (((-1.198196, 44.632016),
      (-1.217422, 44.595846),
      (-1.255875, 44.540569),
      (-1.256561, 44.517562),
      (-1.243515, 44.488668),
      (-1.196823, 44.490628),
      (-1.170731, 44.527353),
      (-1.156998, 44.524906),
      (-1.137772, 44.582643),
      (-1.141891, 44.60709),
      (-1.128845, 44.622732),
      (-1.124039, 44.642277),
      (-1.148758, 44.654001),
      (-1.153564, 44.666211),
      (-1.175537, 44.665234),
      (-1.19545, 44.661816),
      (-1.20575, 44.649605),
      (-1.198196, 44.632016)),)},
   'bbox': (-1.256561, 44.488668, -1.124039, 44.666211)}],
 'bbox': (-1.256561, 44.488668, -1.124039, 44.666211)}

### Image_ID

Let's retrieve the image_ids along with some metadata from our search report file.

In [5]:
# Load search results generated from the archive search notebook.

archive_search_file = Path.cwd()/'SEARCH-REPORT_aoi-teste-de-buch-fire.geojson'
open_archive_search_file = up42.read_vector_file(archive_search_file)

In [12]:
# Let's dig up some metadata from our archove search report

for count, scene in enumerate(open_archive_search_file['features']):
    
    cloud_coverage = scene['properties']['cloudCoverage']
    constellation = scene['properties']['constellation']
    image_id = scene['properties']['id']
    
    print(f"{count} - {image_id} - {constellation} - {cloud_coverage}")

0 - 64f592b0-124e-436b-874b-adc3db7ee74a - pneo - 4.0
1 - 428ba0dc-c7ad-43f4-b27e-79147aac91ef - pneo - 3.8
2 - a1f837d9-219a-4720-aa08-042951cb7f96 - phr - 2.24
3 - bddd0821-f2e7-4bef-8226-914c382fae58 - phr - 2.1
4 - 64a3aa64-f8bd-43bc-a706-970ad5116575 - phr - 5.6
5 - a5706370-237d-4d83-a51e-59f8a1e00d03 - phr - 0.0
6 - 262684f7-7b81-40a9-b4ee-ee704dd9f56a - spot - 0.0
7 - 70230250-6af7-4faa-bf53-f8b681ff8382 - spot - 0.0
8 - cd8e3255-9dac-45f7-bf3e-f69bd3cf71fd - phr - 1.85
9 - b869bd70-dda2-4f0e-931a-71c032b9efd4 - spot - 0.0


---
Let's build a list of all the PHR scenes in our search report file.
We could replace phr by pneo or spot because we originally ran the search against phr, pneo and spot all at once in the catalog-search notebook

In [13]:
image_id_list= []

for scene in open_archive_search_file['features']:
    if scene['properties']['collection'] == 'phr':
        image_id_list.append(scene['properties']['id'])
        
image_id_list

['a1f837d9-219a-4720-aa08-042951cb7f96',
 'bddd0821-f2e7-4bef-8226-914c382fae58',
 '64a3aa64-f8bd-43bc-a706-970ad5116575',
 'a5706370-237d-4d83-a51e-59f8a1e00d03',
 'cd8e3255-9dac-45f7-bf3e-f69bd3cf71fd']

</br>
Back to more serious things, we now have an image_id and have confirmed it is Pleiades (PHR) scene. We have two of the three required parameters that are needed by the construct_order_parameters() fuction. The last parameter is finding the product_id for pleiades reflectance. </br>

### Product ID

In [14]:
# Currently available products

prod_dict = catalog.get_data_products()
prod_dict.keys()

dict_keys(['Near Space Labs - 30cm', 'Capella Space GEC', 'Capella Space GEO', 'Capella Space SICD', 'Capella Space SLC', 'Pléiades Neo', 'SPOT 6/7', 'Pléiades', 'TripleSat', 'WorldDEM Neo', 'NEXTMap One DSM', 'NEXTMap One DTM', 'NEXTMap One Bundle', 'NEXTMap 6 DSM', 'NEXTMap 6 DTM', 'NEXTMap 6 Bundle', 'NEXTMap 10 DSM', 'NEXTMap 10 DTM', 'NEXTMap 10 Bundle'])

</br>From the above options we need to find the product values for key 'Pléiades'

In [15]:
# Find the product dictionary values for key 'Pléiades'

search_key = 'Pléiades'

if search_key in prod_dict.keys():
    pprint(prod_dict[search_key])
    

{'collection': 'phr',
 'data_products': {'Analytic': '4f1b2f62-98df-4c74-81f4-5dce45deee99',
                   'Display': '647780db-5a06-4b61-b525-577a8b68bb54'},
 'host': 'oneatlas'}


</br>As you can see in the json above, the UP42 API currently provides access to two Pleiades products. The Display and Analytic products. Both of these products are orthorectified and delivered in UTM CRS. Today we are interested in building order parameters for the analytic product.</br>
If you need to access to a Primary or Projected product please reach out to the sales team at sales@up42.com or myself at alex.bishop@up42.com
</br>

### Order parameters

We can now bring all our parameters together and pass them to the construct_order_parameters() function. From the list of phr image id's we made, let's see how much the third scene or image_id_list[2] would cost.

In [16]:
# Construct order parameters

single_scene_order_parameters = catalog.construct_order_parameters(
    data_product_id = prod_dict[search_key]['data_products']['Analytic'], 
    image_id = image_id_list[2],
    aoi = aoi)

single_scene_order_parameters

2023-02-09 15:28:43,479 - See `catalog.get_data_product_schema(data_product_id)` for more detail on the parameter options.


{'dataProduct': '4f1b2f62-98df-4c74-81f4-5dce45deee99',
 'params': {'id': '64a3aa64-f8bd-43bc-a706-970ad5116575',
  'aoi': {'type': 'Polygon',
   'coordinates': (((-1.198196, 44.632016),
     (-1.217422, 44.595846),
     (-1.255875, 44.540569),
     (-1.256561, 44.517562),
     (-1.243515, 44.488668),
     (-1.196823, 44.490628),
     (-1.170731, 44.527353),
     (-1.156998, 44.524906),
     (-1.137772, 44.582643),
     (-1.141891, 44.60709),
     (-1.128845, 44.622732),
     (-1.124039, 44.642277),
     (-1.148758, 44.654001),
     (-1.153564, 44.666211),
     (-1.175537, 44.665234),
     (-1.19545, 44.661816),
     (-1.20575, 44.649605),
     (-1.198196, 44.632016)),)}}}

## Get credit estimate

Finaly, we simply pass our order_parameters to the estimate_order() function

In [17]:
# Credit estimate

credit_estimate = catalog.estimate_order(single_scene_order_parameters)
print(f"Credit estimate: {credit_estimate} ({credit_estimate / 100} euros)")

2023-02-09 15:28:51,158 - Order is estimated to cost 98580 UP42 credits (order_parameters: {'dataProduct': '4f1b2f62-98df-4c74-81f4-5dce45deee99', 'params': {'id': '64a3aa64-f8bd-43bc-a706-970ad5116575', 'aoi': {'type': 'Polygon', 'coordinates': (((-1.198196, 44.632016), (-1.217422, 44.595846), (-1.255875, 44.540569), (-1.256561, 44.517562), (-1.243515, 44.488668), (-1.196823, 44.490628), (-1.170731, 44.527353), (-1.156998, 44.524906), (-1.137772, 44.582643), (-1.141891, 44.60709), (-1.128845, 44.622732), (-1.124039, 44.642277), (-1.148758, 44.654001), (-1.153564, 44.666211), (-1.175537, 44.665234), (-1.19545, 44.661816), (-1.20575, 44.649605), (-1.198196, 44.632016)),)}}})


Credit estimate: 98580 (985.8 euros)


---
More fun, we can easily loop over all the scenes in our image_id_list we made earlier and get the credit estimates to all the scenes at once.</br>

In [18]:
for scene in image_id_list:
    order_parameters = catalog.construct_order_parameters(
        data_product_id = prod_dict[search_key]['data_products']['Analytic'], 
        image_id = scene,
        aoi = aoi)
    
    credit_estimate = catalog.estimate_order(order_parameters)
    print(f"Scene {scene} cost {credit_estimate} ({credit_estimate / 100} euros)")

2023-02-09 15:28:53,476 - See `catalog.get_data_product_schema(data_product_id)` for more detail on the parameter options.
2023-02-09 15:28:56,069 - Order is estimated to cost 114690 UP42 credits (order_parameters: {'dataProduct': '4f1b2f62-98df-4c74-81f4-5dce45deee99', 'params': {'id': 'a1f837d9-219a-4720-aa08-042951cb7f96', 'aoi': {'type': 'Polygon', 'coordinates': (((-1.198196, 44.632016), (-1.217422, 44.595846), (-1.255875, 44.540569), (-1.256561, 44.517562), (-1.243515, 44.488668), (-1.196823, 44.490628), (-1.170731, 44.527353), (-1.156998, 44.524906), (-1.137772, 44.582643), (-1.141891, 44.60709), (-1.128845, 44.622732), (-1.124039, 44.642277), (-1.148758, 44.654001), (-1.153564, 44.666211), (-1.175537, 44.665234), (-1.19545, 44.661816), (-1.20575, 44.649605), (-1.198196, 44.632016)),)}}})
2023-02-09 15:28:56,070 - See `catalog.get_data_product_schema(data_product_id)` for more detail on the parameter options.


Scene a1f837d9-219a-4720-aa08-042951cb7f96 cost 114690 (1146.9 euros)


2023-02-09 15:28:58,362 - Order is estimated to cost 115030 UP42 credits (order_parameters: {'dataProduct': '4f1b2f62-98df-4c74-81f4-5dce45deee99', 'params': {'id': 'bddd0821-f2e7-4bef-8226-914c382fae58', 'aoi': {'type': 'Polygon', 'coordinates': (((-1.198196, 44.632016), (-1.217422, 44.595846), (-1.255875, 44.540569), (-1.256561, 44.517562), (-1.243515, 44.488668), (-1.196823, 44.490628), (-1.170731, 44.527353), (-1.156998, 44.524906), (-1.137772, 44.582643), (-1.141891, 44.60709), (-1.128845, 44.622732), (-1.124039, 44.642277), (-1.148758, 44.654001), (-1.153564, 44.666211), (-1.175537, 44.665234), (-1.19545, 44.661816), (-1.20575, 44.649605), (-1.198196, 44.632016)),)}}})
2023-02-09 15:28:58,363 - See `catalog.get_data_product_schema(data_product_id)` for more detail on the parameter options.


Scene bddd0821-f2e7-4bef-8226-914c382fae58 cost 115030 (1150.3 euros)


2023-02-09 15:29:01,498 - Order is estimated to cost 98580 UP42 credits (order_parameters: {'dataProduct': '4f1b2f62-98df-4c74-81f4-5dce45deee99', 'params': {'id': '64a3aa64-f8bd-43bc-a706-970ad5116575', 'aoi': {'type': 'Polygon', 'coordinates': (((-1.198196, 44.632016), (-1.217422, 44.595846), (-1.255875, 44.540569), (-1.256561, 44.517562), (-1.243515, 44.488668), (-1.196823, 44.490628), (-1.170731, 44.527353), (-1.156998, 44.524906), (-1.137772, 44.582643), (-1.141891, 44.60709), (-1.128845, 44.622732), (-1.124039, 44.642277), (-1.148758, 44.654001), (-1.153564, 44.666211), (-1.175537, 44.665234), (-1.19545, 44.661816), (-1.20575, 44.649605), (-1.198196, 44.632016)),)}}})
2023-02-09 15:29:01,500 - See `catalog.get_data_product_schema(data_product_id)` for more detail on the parameter options.


Scene 64a3aa64-f8bd-43bc-a706-970ad5116575 cost 98580 (985.8 euros)


2023-02-09 15:29:04,061 - Order is estimated to cost 35190 UP42 credits (order_parameters: {'dataProduct': '4f1b2f62-98df-4c74-81f4-5dce45deee99', 'params': {'id': 'a5706370-237d-4d83-a51e-59f8a1e00d03', 'aoi': {'type': 'Polygon', 'coordinates': (((-1.198196, 44.632016), (-1.217422, 44.595846), (-1.255875, 44.540569), (-1.256561, 44.517562), (-1.243515, 44.488668), (-1.196823, 44.490628), (-1.170731, 44.527353), (-1.156998, 44.524906), (-1.137772, 44.582643), (-1.141891, 44.60709), (-1.128845, 44.622732), (-1.124039, 44.642277), (-1.148758, 44.654001), (-1.153564, 44.666211), (-1.175537, 44.665234), (-1.19545, 44.661816), (-1.20575, 44.649605), (-1.198196, 44.632016)),)}}})
2023-02-09 15:29:04,063 - See `catalog.get_data_product_schema(data_product_id)` for more detail on the parameter options.


Scene a5706370-237d-4d83-a51e-59f8a1e00d03 cost 35190 (351.9 euros)


2023-02-09 15:29:06,002 - Order is estimated to cost 73660 UP42 credits (order_parameters: {'dataProduct': '4f1b2f62-98df-4c74-81f4-5dce45deee99', 'params': {'id': 'cd8e3255-9dac-45f7-bf3e-f69bd3cf71fd', 'aoi': {'type': 'Polygon', 'coordinates': (((-1.198196, 44.632016), (-1.217422, 44.595846), (-1.255875, 44.540569), (-1.256561, 44.517562), (-1.243515, 44.488668), (-1.196823, 44.490628), (-1.170731, 44.527353), (-1.156998, 44.524906), (-1.137772, 44.582643), (-1.141891, 44.60709), (-1.128845, 44.622732), (-1.124039, 44.642277), (-1.148758, 44.654001), (-1.153564, 44.666211), (-1.175537, 44.665234), (-1.19545, 44.661816), (-1.20575, 44.649605), (-1.198196, 44.632016)),)}}})


Scene cd8e3255-9dac-45f7-bf3e-f69bd3cf71fd cost 73660 (736.6 euros)


---
Nice ! Next we will see how to order the data. For this we will need the order_parameters we just built so let's export that to a simple geojson file.


In [19]:
import json

with open("order_parameters_"+geojson_aoi_file.stem+".json", "w") as outfile:
    json.dump(single_scene_order_parameters, outfile)

## Documentation

https://sdk.up42.com/reference/catalog-reference

Happy coding ! I hope this short notebook was heplfull, if you have any question please feel free to reach out to me at alex.bishop@up42.com or support@up42.com